In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)


In [2]:
spark

In [23]:
def splitNeighbours(parentPath,parentDist,neighbour):
    
    nid, weight = neighbour.split(',')
    dist = int(weight) + parentDist
    path = parentPath + '->' + nid
    
    return (nid, (dist,'None',path))
    
        
        
        

In [11]:
def minDistance(x,y):
    z = [0,0,0]
    
    if x[0] < y[0]:
        z[0] = x[0]
        z[2] = x[2]
        
    else:
        z[0] = y[0]
        z[2] = y[2]
    
    if x[1] == 'None':
        z[1] = y[1]
        
    else:
        z[1] = x[1]
    
    return tuple(z)
    
        

In [31]:
rdd1 = sc.parallelize([('1', (0,['2,10','3,5'],'1')),('2', (999,['3,2','4,1'],'2')),('3', (999,['2,3','4,9','5,2'],'3')),
                    ('4', (999,['5,4'],'4')),('5', (999,['1,7','4,6'],'5'))])
n = 0

while n < 4 :
    
    rdd2 = rdd1.map(lambda x: x[1])
    rdd3 = rdd2.map(lambda nodeData: map(lambda neighbour: splitNeighbours(nodeData[2],nodeData[0],neighbour),nodeData[1])).flatMap(lambda x: x)
    rdd4 = rdd1.union(rdd3)
    rdd5 = rdd4.reduceByKey(lambda x,y: minDistance(x,y))
    rdd1 = rdd5
    n = n + 1

rdd1.collect()

[('4', (9, ['5,4'], '1->3->2->4')),
 ('5', (7, ['1,7', '4,6'], '1->3->5')),
 ('1', (0, ['2,10', '3,5'], '1')),
 ('3', (5, ['2,3', '4,9', '5,2'], '1->3')),
 ('2', (8, ['3,2', '4,1'], '1->3->2'))]